In [2]:
import pandas as pd
import json
from math import log
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', 300)

In [27]:
quarters = [20171231, 20180331, 20180630, 20180930, 20181231, 20190331]
splc_all = pd.read_excel("Supply Chain Data All.xlsx", sheet_name="Supply Chain")
splc_all.head()

,Central Ticker,Central Company,As Of Date Bloomberg,Central Country,Market_Cap_Central,Stock Number,Listed Country,Supplier Registered Name,Supplier Company,Supplier Ticker,Relationship Type,Supplier Country,Supplier Market Cap,Latest Inv. Growth,% Revenue get from central,Relationship Value,Currency,Account As Type,%Cost,Relationship Value USD,Source,As Of Date
0,1188 HK Equity,Hybrid Kinetic,20181231,US,93.400636,NaN,--,--,Anhui Tiankang HK New Energy Technology Co Ltd,1471398D CH Equity,Suppliers,CN,0.000000e+00,--,NaN,39.94k,USD,COGS,0.0479,39940.0,2017A CF,2018-04-27 00:00:00
1,000572 CH Equity,Haima Automobi-A,20181231,CN,509.279000,1512,TT,JUI LI ENTERPRISE CO LTD,JUI LI Enterprise Co Ltd,1512 TT Equity,Suppliers,TW,3.271000e+07,--,0.1700,3.55M,USD,COGS,0.0127,3550000.0,*2017A CF,2018-06-14 00:00:00
2,000572 CH Equity,Haima Automobi-A,20181231,CN,509.279000,ALV,US,AUTOLIV INC,Autoliv Inc,ALV US Equity,Suppliers,SE,6.510000e+09,--,0.0003,956.41k,USD,COGS,0.0029,956410.0,Estimate,2018-06-20 00:00:00
3,000572 CH Equity,Haima Automobi-A,20181231,CN,509.279000,NaN,--,--,Haima Investment Group Corp,HAIMAZ CH Equity,Suppliers,CN,0.000000e+00,--,NaN,818.84k,USD,COGS,0.0025,818840.0,2017A CF,2018-04-02 00:00:00
4,000572 CH Equity,Haima Automobi-A,20181231,CN,509.279000,CPS,US,COOPER-STANDARD HOLDING,Cooper-Standard Holdings Inc,CPS US Equity,Suppliers,US,9.206300e+08,--,0.0002,159.95k,USD,COGS,0.0005,159950.0,Estimate,2018-07-25 00:00:00


In [4]:
central_ticker_list = splc_all['Central Ticker'].unique().tolist()
central_ticker_list

['ZAAP US Equity',
 'TSLA US Equity',
 'HOG US Equity',
 'GM US Equity',
 'F US Equity',
 '601127 CH Equity',
 '000572 CH Equity',
 '000800 CH Equity',
 '600418 CH Equity',
 '000913 CH Equity',
 '000927 CH Equity',
 '600609 CH Equity',
 '1211 HK Equity',
 '175 HK Equity',
 '1958 HK Equity',
 '200054 CH Equity',
 '200625 CH Equity',
 '2238 HK Equity',
 '2333 HK Equity',
 '600877 CH Equity',
 '603766 CH Equity',
 '489 HK Equity',
 '600006 CH Equity',
 '601965 CH Equity',
 '600099 CH Equity',
 '600104 CH Equity',
 '200550 CH Equity',
 '600166 CH Equity',
 '000980 CH Equity',
 '1122 HK Equity',
 '600303 CH Equity',
 'THO US Equity',
 'WGO US Equity',
 '601777 CH Equity',
 '600733 CH Equity',
 '1188 HK Equity',
 '603611 CH Equity',
 'NIO US Equity']

In [5]:
region = pd.read_excel("Supply_Chain_Data_20190510.xlsx", sheet_name="Country Code")
region.head()

,Country Code,Country Name,Region
0,AE,United Arab Emirates,Middle East
1,AR,Argentina,Latin America
2,AT,Austria,Europe
3,AU,Australia,APAC
4,BE,Belgium,Europe


In [38]:
# scores
scores_sheet = pd.read_excel("abnormal financial change fangyi.xlsx")
scores = {}

for _, info in scores_sheet.iterrows():
    ticker = info["Tickers"]
    if ticker not in scores:
        scores[ticker] = {"Risk Score": [], "Borrow Need": [], "FX Need": []}
    if pd.isna(info["Borrow Need"]): info["Borrow Need"] = 0
    if pd.isna(info["FX Need"]): info["FX Need"] = 0
    if pd.isna(info["Risk Score"]): info["Risk Score"] = 0
    scores[ticker]["Borrow Need"].append(info["Borrow Need"])
    scores[ticker]["Risk Score"].append(info["Risk Score"])
    scores[ticker]["FX Need"].append(info["FX Need"])

#print(scores["000120 KS Equity"])
for k, v in scores.items():
    try:
        assert(len(v["Borrow Need"]) == 4)
        assert(len(v["FX Need"]) == 4)
        assert(len(v["Risk Score"]) == 4)
    except:
        print(k)

In [42]:
data_centrality = pd.read_excel("Impact Score(4).xlsx")
centrality_hub = {}
centrality_auth = {}

for i in range(6):
    comp_q_data = data_centrality[(data_centrality["date"] == quarters[i])]
    print(len(comp_q_data))
    for _, info in comp_q_data.iterrows():
        if info["Ticker"] not in centrality_hub:
            centrality_hub[info["Ticker"]] = {}
            centrality_auth[info["Ticker"]] = {}
        centrality_hub[info["Ticker"]][i] = info["Hub score"] / 0.003956894201773358
        centrality_auth[info["Ticker"]][i] = info["Auth score"] / 0.04582665265807497

4460
4318
4063
3923
3923
3965


In [22]:
number_of_sup = json.load(open("numofsupplier.json"))

In [6]:
region_dict = {}
for _, info in region.iterrows():
    region_dict[info["Country Code"]] = info["Region"]
print(region_dict)

{'AE': 'Middle East', 'AR': 'Latin America', 'AT': 'Europe', 'AU': 'APAC', 'BE': 'Europe', 'BR': 'Latin America', 'CA': 'North America', 'CH': 'Europe', 'CL': 'Latin America', 'CN': 'China', 'CO': 'Latin America', 'CY': 'Middle East', 'DE': 'Europe', 'DK': 'Europe', 'ES': 'Europe', 'FI': 'Europe', 'FR': 'Europe', 'GB': 'Europe', 'HK': 'Hong Kong', 'HR': 'Europe', 'ID': 'APAC', 'IE': 'Europe', 'IL': 'Middle East', 'IN': 'APAC', 'IT': 'Europe', 'JP': 'APAC', 'KR': 'APAC', 'KY': 'Europe', 'LK': 'APAC', 'LU': 'Europe', 'MX': 'North America', 'MY': 'APAC', 'NL': 'Europe', 'NO': 'Europe', 'PE': 'Latin America', 'PH': 'APAC', 'PK': 'Middle East', 'RO': 'Europe', 'RU': 'Europe', 'SA': 'Middle East', 'SE': 'Europe', 'SG': 'APAC', 'TH': 'APAC', 'TR': 'Middle East', 'TW': 'China', 'US': 'USA', 'VN': 'APAC', 'ZA': 'Africa', 'QA': 'Middle East', 'MO': 'China'}


In [55]:
nodes = {}
links = {}

def set_default_n(node):
    node['centrality_score'] = {}
    node['size'] = {}
    node['market_cap'] = {}
    node['borrow'] = {}
    node['risk'] = {}
    node['FX'] = {}
    node['num_supplier'] = {}
    for i in range(6):
        node['centrality_score'][i] = 0
        node['size'][i] = 3
        node['market_cap'][i] = '--'
        node['borrow'][i] = -0.001
        node['risk'][i] = -0.001
        node['FX'][i] = -0.001
        node['num_supplier'][i] = '--'
        
def set_default_l(link):
    link['score'] = {}
    for i in range(6):
        link['score'][i] = 1

for i in range(6):
    comp_q_data = splc_all[(splc_all["As Of Date Bloomberg"] == quarters[i])]
    for _, info in comp_q_data.iterrows():
        if info['Relationship Type'] == 'Customers': continue
        link_id = info['Supplier Ticker'] + "_" + info['Central Ticker']
        if link_id not in links:
            link = {}
            link['source'] = info['Supplier Ticker']
            link['target'] = info['Central Ticker']
            set_default_l(link)
            links[link_id] = link
        else:
            link = links[link_id]

        relation_value = info["Relationship Value USD"]
        link["score"][i] = max(log(relation_value + 1) / 5 - 1, 1)

        if info['Supplier Ticker'] not in nodes:
            node = {}
            node['ticker'] = info['Supplier Ticker']
            node['name'] = info['Supplier Company']
            node['type'] = "supplier1"
            country = info['Supplier Country']
            node['country'] = country
            if country in region_dict:
                node['region'] = region_dict[country]
            else:
                node['country'] = "CN"
                node['region'] = "China"
            set_default_n(node)
            nodes[info['Supplier Ticker']] = node
        else:
            node = nodes[info['Supplier Ticker']]

        market_cap = info['Supplier Market Cap']
        if market_cap == '#N/A' or market_cap == '--' or market_cap == 0 or pd.isna(market_cap):
            node["size"][i] = 5
            node["market_cap"][i] = "--"
        else:
            node["size"][i] = max(log(market_cap) - 12, 5)
            node["market_cap"][i] = market_cap
        
        if i > 1 and (node['ticker'] in scores):
            node['borrow'][i] = scores[node['ticker']]['Borrow Need'][i - 2]
            node['risk'][i] = scores[node['ticker']]['Risk Score'][i - 2]
            node['FX'][i] = scores[node['ticker']]['FX Need'][i - 2]
        
        if i in centrality_hub[node['ticker']]:
            node["centrality_score"][i] = centrality_hub[node['ticker']][i]
        else:
            node["centrality_score"][i] = -1
        node['num_supplier'][i] = number_of_sup[node['ticker']][i]

        # also add the central node
        if info['Central Ticker'] not in nodes:
            node = {}
            node['ticker'] = info['Central Ticker']
            node['name'] = info['Central Company']
            node['type'] = "manufacturer"
            set_default_n(node)

            country = info['Central Country']
            node['country'] = country
            if country in region_dict:
                node['region'] = region_dict[country]
            else:
                node['country'] = "CN"
                node['region'] = "China"
            nodes[info['Central Ticker']] = node
        else:
            node = nodes[info['Central Ticker']]

        if market_cap == '#N/A' or market_cap == '--' or market_cap == 0 or pd.isna(market_cap):
            node["size"][i] = 5
            node["market_cap"][i] = "--"
        else:
            node["size"][i] = max(log(market_cap) - 12, 5)
            node["market_cap"][i] = market_cap


        if i > 1 and (node['ticker'] in scores):
            node['borrow'][i] = scores[node['ticker']]['Borrow Need'][i - 2]
            node['risk'][i] = scores[node['ticker']]['Risk Score'][i - 2]
            node['FX'][i] = scores[node['ticker']]['FX Need'][i - 2]

        if i in centrality_auth[node['ticker']]:
            node["centrality_score"][i] = centrality_auth[node['ticker']][i]
        else:
            node["centrality_score"][i] = -1
        node['num_supplier'][i] = number_of_sup[node['ticker']][i]

In [56]:
json.dump({'nodes': list(nodes.values()), 'links': list(links.values())}, open("splc_six_quarters.json", 'w'), indent=2)

{'Zotye New... Equity': [0, 0, 0, 0, 0, 0],
 '7235 JP Equity': [2, 2, 2, 2, 2, 0],
 '002400 CH Equity': [4, 7, 6, 6, 6, 6],
 '300432 CH Equity': [1, 2, 5, 5, 5, 5],
 'FAW Asset... Equity': [0, 0, 0, 0, 0, 0],
 'PVL FP Equity': [4, 4, 3, 3, 2, 2],
 '8106906Z CH Equity': [0, 0, 0, 0, 0, 0],
 '000957 CH Equity': [0, 0, 0, 0, 0, 0],
 '5482 JP Equity': [8, 8, 8, 7, 9, 9],
 'SAT TB Equity': [0, 0, 0, 0, 0, 0],
 '0478818D CH Equity': [0, 0, 0, 0, 0, 0],
 '8087574Z CH Equity': [0, 1, 1, 1, 1, 1],
 '300256 CH Equity': [8, 8, 6, 6, 6, 5],
 '600742 CH Equity': [30, 0, 18, 18, 18, 17],
 'Jiangxi J5... Equity': [0, 0, 0, 0, 0, 0],
 'CPG LN Equity': [10, 9, 7, 8, 10, 10],
 'Hefei Jia2... Equity': [0, 0, 0, 0, 0, 0],
 '2824482Z US Equity': [0, 0, 0, 0, 0, 0],
 '7202 JP Equity': [103, 102, 100, 102, 0, 100],
 'FAW Bus C... Equity': [0, 0, 0, 0, 0, 0],
 '7244 JP Equity': [0, 0, 0, 0, 0, 0],
 'Shenyang7 ... Equity': [0, 0, 0, 0, 0, 0],
 '6901 JP Equity': [0, 0, 0, 0, 0, 0],
 '1293798D CH Equity': [1, 1,